In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
from domino.data.celeb import build_celeb_df, get_celeb_dp
from mosaic import ImageColumn
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import wandb
wandb.login()
from domino.vision import train

[2021-06-09 21:55:20,853][ERROR][wandb.jupyter:144] :: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hazy-research (use `wandb login --relogin` to force relogin)


In [3]:
#df = build_celeb_df()
df = build_celeb_df.out(473)

In [4]:
dp = get_celeb_dp(df.load())

In [1]:
from domino.utils import PredLogger
logger = PredLogger()

In [5]:
logger

PredLogger()

In [7]:
train(
    dp=dp,
    input_column="input",
    id_column="file",
    target_column="bald",
    batch_size=32, 
    num_worker=2, 
    run_dir="test"
)

[2021-06-09 21:56:15,341][INFO][pytorch_lightning.utilities.seed:65] :: Global seed set to 123
[2021-06-09 21:56:15,668][INFO][pytorch_lightning.utilities.distributed:73] :: GPU available: True, used: True
[2021-06-09 21:56:15,669][INFO][pytorch_lightning.utilities.distributed:73] :: TPU available: False, using: 0 TPU cores
[2021-06-09 21:56:15,684][INFO][pytorch_lightning.accelerators.gpu:63] :: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[2021-06-09 21:56:15,704][INFO][pytorch_lightning.core.lightning:1496] :: 
  | Name        | Type       | Params
-------------------------------------------
0 | model       | ResNet     | 11.2 M
1 | metrics     | ModuleDict | 0     
2 | valid_preds | PredLogger | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


ValueError: No negative samples in targets, false positive value should be meaningless

In [19]:
from domino.evaluate.linear import induce_correlation
from mosaic.contrib.celeba import ATTRIBUTES

In [42]:
np.linspace(0, 0.95, 10)

array([0.        , 0.10555556, 0.21111111, 0.31666667, 0.42222222,
       0.52777778, 0.63333333, 0.73888889, 0.84444444, 0.95      ])

In [41]:
results = []
for attr_a in tqdm(ATTRIBUTES):
    for attr_b in ATTRIBUTES:
        for corr in np.linspace(0, 0.95, 20):
            try:
                induce_correlation(
                    dp, 
                    corr=corr, 
                    attr_a=attr_a, 
                    attr_b=attr_b, 
                    n=20000, 
                    match_mu=True
                )
                success = True
            except ValueError:
                success = False
            results.append({
                "attr_a": attr_a,
                "attr_b": attr_b,
                "corr": corr,
                "success": success
            }) 


In [43]:
df = pd.DataFrame(results)
df = df.groupby(["attr_a", "attr_b"]).success.mean().reset_index()
df.pivot(index="attr_a", columns="attr_b")

success                             \
attr_b              5_o_clock_shadow arched_eyebrows attractive   
attr_a                                                            
5_o_clock_shadow                0.00            0.70       1.00   
arched_eyebrows                 0.70            0.00       1.00   
attractive                      1.00            1.00       0.00   
bags_under_eyes                 1.00            1.00       1.00   
bald                            1.00            0.65       0.35   
bangs                           0.60            1.00       1.00   
big_lips                        1.00            1.00       1.00   
big_nose                        1.00            1.00       1.00   
black_hair                      1.00            1.00       1.00   
blond_hair                      0.05            1.00       1.00   
blurry                          0.70            1.00       1.00   
brown_hair                      1.00            1.00       1.00   
bushy_eyebrows                  1.00            1.00       1.00   
chubby                          1.00            1.00       0.30   
double_chin                     1.00            1.00       0.40   
eyeglasses                      1.00            0.15       0.95   
goatee                          1.00            0.75       1.00   
gray_hair                       0.45            0.60       0.30   
heavy_makeup                    0.00            1.00       1.00   
high_cheekbones                 1.00            1.00       1.00   
male                            0.00            0.80       1.00   
mouth_slightly_open             1.00            1.00       1.00   
mustache                        1.00            0.85       1.00   
narrow_eyes                     1.00            1.00       1.00   
no_beard                        0.75            0.55       1.00   
oval_face                       1.00            1.00       1.00   
pale_skin                       0.55            1.00       1.00   
pointy_nose                     1.00            1.00       1.00   
receding_hairline               0.90            1.00       1.00   
rosy_cheeks                     0.00            1.00       1.00   
sideburns                       1.00            0.60       1.00   
smiling                         1.00            1.00       1.00   
straight_hair                   1.00            1.00       1.00   
wavy_hair                       1.00            1.00       1.00   
wearing_earrings                0.00            1.00       1.00   
wearing_hat                     1.00            0.75       1.00   
wearing_lipstick                0.00            1.00       1.00   
wearing_necklace                0.05            1.00       1.00   
wearing_necktie                 1.00            0.55       1.00   
young                           1.00            1.00       1.00   

                                                                              \
attr_b              bags_under_eyes  bald bangs big_lips big_nose black_hair   
attr_a                                                                         
5_o_clock_shadow                1.0  1.00  0.60      1.0     1.00       1.00   
arched_eyebrows                 1.0  0.65  1.00      1.0     1.00       1.00   
attractive                      1.0  0.35  1.00      1.0     1.00       1.00   
bags_under_eyes                 0.0  1.00  1.00      1.0     1.00       1.00   
bald                            1.0  0.00  0.00      1.0     1.00       0.15   
bangs                           1.0  0.00  0.00      1.0     1.00       1.00   
big_lips                        1.0  1.00  1.00      0.0     1.00       1.00   
big_nose                        1.0  1.00  1.00      1.0     0.00       1.00   
black_hair                      1.0  0.15  1.00      1.0     1.00       0.00   
blond_hair                      1.0  0.00  1.00      1.0     0.75       0.00   
blurry                          1.0  0.40  1.00      1.0     1.00       1.00   
brown_hair                 

In [53]:
np.linspace(0, 0.95, 20)

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

In [44]:
results = []
for attr_a in tqdm(ATTRIBUTES):
    for attr_b in ATTRIBUTES:
        for corr in np.linspace(0, 0.95, 20):
            try:
                induce_correlation(
                    dp, 
                    corr=corr, 
                    attr_a=attr_a, 
                    attr_b=attr_b, 
                    n=20000, 
                    match_mu=False
                )
                success = True
            except ValueError:
                success = False
            results.append({
                "attr_a": attr_a,
                "attr_b": attr_b,
                "corr": corr,
                "success": success
            }) 


In [52]:
df = pd.DataFrame(results)
df = df.groupby(["attr_a", "attr_b"]).success.mean().reset_index()
df[df.success >0.9]

,attr_a,attr_b,success
79,arched_eyebrows,young,0.909091
99,attractive,high_cheekbones,0.909091
100,attractive,male,0.909091
101,attractive,mouth_slightly_open,1.000000
111,attractive,smiling,1.000000
116,attractive,wearing_lipstick,1.000000
144,bags_under_eyes,no_beard,0.909091
159,bags_under_eyes,young,0.909091
279,big_lips,young,1.000000
319,big_nose,young,1.000000


In [50]:
results = []
for attr_a in tqdm(ATTRIBUTES):
    for attr_b in ATTRIBUTES:
        for corr in np.linspace(-0.95, 0, 11):
            try:
                induce_correlation(
                    dp, 
                    corr=corr, 
                    attr_a=attr_a, 
                    attr_b=attr_b, 
                    n=20000, 
                    match_mu=False
                )
                success = True
            except ValueError:
                success = False
            results.append({
                "attr_a": attr_a,
                "attr_b": attr_b,
                "corr": corr,
                "success": success
            }) 


In [51]:
df = pd.DataFrame(results)
df = df.groupby(["attr_a", "attr_b"]).success.mean().reset_index()
df[df.success == 1]

,attr_a,attr_b,success
101,attractive,mouth_slightly_open,1.0
111,attractive,smiling,1.0
116,attractive,wearing_lipstick,1.0
279,big_lips,young,1.0
319,big_nose,young,1.0
359,black_hair,young,1.0
842,mouth_slightly_open,attractive,1.0
1242,smiling,attractive,1.0
1442,wearing_lipstick,attractive,1.0
1566,young,big_lips,1.0


In [61]:
from torch.utils.data import DataLoader
dl = DataLoader(dp[["input", "file", "bald"]].lz[dp["split"].data == "valid"])

In [62]:
dp.write()

In [71]:
from domino.vision import train
train(
    dp, 
    input_column="input",
    target_column="bald",
    id_column="file",
)

[2021-06-09 21:01:30,401][INFO][pytorch_lightning.utilities.seed:65] :: Global seed set to 123
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


TypeError: super(type, obj): obj must be an instance or subtype of type